In [4]:
# !pip install google-search-results
# !pip install numexpr
# !pip install "langchain[all]"

Let's take a look at a simple example:

In [5]:
from dotenv import load_dotenv

load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
# os.environ["OPENAI_API_KEY"] = "your-api-key"

In [6]:
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, AgentExecutor

llm = ChatOpenAI(temperature=0)
# you need to set up your SERP_API_KEY in your environment variables
tools = load_tools(["serpapi", "llm-math"], llm=llm)

prompt = hub.pull("hwchase17/react")

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

llm_with_stop = llm.bind(stop=["\nObservation"])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...
I need to find out who Leo DiCaprio's girlfriend is and then calculate her current age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"Ever since Vittoria Ceretti was first romantically linked to Leonardo DiCaprio in 2023, public interest in the model has soared. Ceretti, 25, isn't new to the modeling scene, and she has had a successful career on the runway for more than a decade. Here's everything to know about Ceretti.Now I need to find out Vittoria Ceretti's current age.
Action: Calculator
Action Input: 25^0.43Answer: 3.991298452658078I now know the final answer.
Final Answer: 3.99

> Finished chain.


{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': '3.99'}

• ```llm = ChatOpenAI(temperature=0)```: Create an instance of the ChatOpenAI language model with a temperature of 0.

• ```tools = load_tools(["serpapi", "llm-math"], llm=llm)```: Load the tools "serpapi" and "llm-math" with the specified language model.

• ```prompt = hub.pull("hwchase17/react")```: Retrieve the prompt from the hub with the name "hwchase17/react".

• ```prompt = prompt.partial(tools=render_text_description(tools), tool_names=", ".join([t.name for t in tools]))```: Modify the prompt by rendering the text description of the tools and joining their names.

• ```llm_with_stop = llm.bind(stop=["\nObservation"])```: Bind the language model with a stop condition set to the token "\nObservation".

• ```agent = (...)```: Define the agent using a series of steps, including input mapping, prompt, language model with stop condition, and output parsing.

• ```agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)```: Create an instance of the AgentExecutor with the defined agent and tools.

• ```agent_executor.invoke(...)```: Invoke the agent by passing an input to it and execute its chain of actions and tools.